# Demo script

### Part 1 - Fivetran

##### Step 1: Set up Redshift as the destination data warehouse

* Log in to https://fivetran.com as an administrator
* Select `Manage account` from the top left menu, or navigate your browser to https://fivetran.com/account
* From the Destinations panel, add a new destination
* Choose `Redshift` as the warehouse type. Fill in the details as shown below

<img src="media/fivetran-warehouse.png" width="500px">

##### Step 2: Define connectors for each data source

* On the left pane, select Connectors
* Add a new connector for Google Ads. Follow the instructions to connect to your Google Ads account
* In this demo we will use Google Sheets instead of Google Ads. Ensure that the [spreadsheet](https://docs.google.com/spreadsheets/d/1BzGIzwDvyIDEeMsGj8wC1EXxEAEPodf8U-BUXKuZp-w/edit#gid=0) is shared with the [service account](mailto:g-airplane-chlorophyll@fivetran-production.iam.gserviceaccount.com) for your Fivetran account
* Set the schema to `google_ads_demo`. Set the destination table name to match the tab name, and the Sheet URL to point to your spreadsheet. Pick the named range from the dropdown menu (eg. `final_url_performance`)
* Save and test the connection

<img src="media/fivetran-connectors.png" width="500px">

##### Step 3: Set up synchronization schedule

* Open each connection and set the sync frequency (eg. every 6 hours)
* Optionally, select `Sync now` to perform a manual sync

### Part 2 - Trifacta

##### Step 1: Apply data quality rules

In the `final_url_summary`, we want to view metrics for each `date`, `campaign` and `ad group`. To ensure accurate reporting, we apply a data quality rule to check if the combination of `date`, `campaign_name` and `ad_group_name` is unique.
* Merge the 3 columns: `date`, `campaign_name` and `ad_group_name`
* Create a data quality rule to check if the combinations are unique

Based on reviewing the data quality rule, it looks like we have some duplicates.

![trifacta-dq-unique.png](media/trifacta-dq-unique.png)

Let's resolve these duplicates by summarizing the data. We will use a pivot table to group values by `date_campaign_ad_group`, `date`, `campaign_name` and `ad_group_name`. Calculate the sum of `cost`, `impressions` and `clicks`. After that, check the data quality rule again. This time it should pass.

##### Step 2: Build time series

We want to ensure that we have a full 3 quarters worth of data, with no gaps in between. To do this, we create a time series covering the first 3 quarters of 2020. We invoke a `TimeSeries` macro that accepts `start_date`, `end_date` and `new_column_name` as parameters. 

![media/trifacta-macro-timeseries.png](media/trifacta-macro-timeseries.png)

In a single step, we have created a range of dates. Behind the scenes, this executes a series of steps (show macro `Inspect` view).

<img src="media/trifacta-macro-inspect.png" width="500px">

Delete the `Field1` column as we don't need it.

##### Step 3: Join and summarize data

We will now combine the time series data with click_performance. Create a new `click_details` recipe starting from the time series. Then perform a lookup based on `date`. Apply a data quality rule to see if there are any missing values for `campaign_name` or `ad_group_name`.

We notice that there is missing data for 6 days.

![media/trifacta-dq-time-series-gaps.png](media/trifacta-dq-time-series-gaps.png)

We won't take any action at this time to address the missing data. The data profile will capture a record of this data quality issue for further investigation.

Switch to the `campaign_summary` recipe. We will summarize campaign metrics to identify the most effective campaigns. For this, we use a pivot table and group by `campaign_name` to sum `cost`, `impressions` and `clicks`

![media/trifacta-pivot.png](media/trifacta-pivot.png)

##### Step 4: Calculate derived metrics

We now have some basic metrics summarized at the campaign level. Let's calculate some derived metrics as follows:

* `cost_per_click` = `sum_cost` / `sum_clicks`
* `impressions_per_click` = `sum_impressions` / `sum_clicks`

To measure effectiveness of marketing spend, we bring in Sales data from our ERP systems (`sales_summary` table) by doing a lookup against the `campaign_name` field. Using this we can calculate `sales_per_marketing_dollar` as follows:

* `sales_per_marketing_dollar` = `sales` / `sum_cost`

Looking at these numbers, we learn that **holiday promotions** are delivering the best bang for our buck.

##### Step 5: Publish results to Redshift

The final step is to publish the prepared data to Redshift. We do this be defining publish destinations and specifying the schema and table where we want to persist the outputs.

![media/trifacta-publish.png](media/trifacta-publish.png)

After specifying the publish destinations, run a job to produce the outputs. The job also produces an output profile for each dataset that provides a data quality score at the end of the process. 

(Show the output profile)

If you are comfortable with the data quality, you can proceed to consume this data through a BI dashboard in Amazon QuickSight.